In [1]:
# RPA Week #4 challenge – https://developer.automationanywhere.com/blog/challenge-page-introaccountspayablechallenge

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import rpa as r
import pandas as pd
import time
import os
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
#============================================
# Functions for data extraction
#============================================

def format1(data, FileName, DataFrame):
    #Extract Invoice Number
    start = data.find('Invoice no.') + len('Invoice no.') 
    end = data.find('Invoice Date')
    content = data[start:end]
    if len(content) > 0:
        content = content.split('\n')
        InvoiceNumber = content[1].split(' ')[-1].strip()
    else:
        print('[{}] InvoiceNumber: Extraction Failed'.format(FileName))
        
    #Extract Invoice Date
    start = data.find('Invoice Date') + len('Invoice Date') 
    end = data.find('Purchase Order')
    content = data[start:end]
    if len(content) > 0:
        content = content.split('\n')
        InvoiceDate = content[1].split(' ')[-3:]
        InvoiceDate = ' '.join(InvoiceDate)
    else:
        print('[{}] InvoiceDate: Extraction Failed'.format(FileName))
        
    #Extract Invoice Total
    start = data.find('Invoice Amount') + len('Invoice Amount') 
    end = data.find('Terms')
    content = data[start:end]
    if len(content) > 0:
        content = content.split('\n')
        InvoiceTotal = content[0].split(' ')[-1].replace(',','').replace('$','')
    else:
        print('[{}] InvoiceTotal: Extraction Failed'.format(FileName))
        
    #Extract Line Items
    start = data.find('Tax Amount') + len('Tax Amount') 
    end = data.find('Subtotal')
    content = data[start:end].replace('|','').replace(']','').replace('}','').replace('  ',' ')

    if len(content) > 0:
        content = content.split('\n')
        for item in content:
            if len(item) > 0:
                item = item.split(' ')

                #Extract Line item - Quantity
                Quantity = item[0].strip()

                #Extract Line item - Item No
                ItemNo = item[1].strip()

                #Extract Line item - Desciption
                Desciption = item[2:-3]
                Desciption = ' '.join(Desciption).strip()

                #Extract Line item - Unit Price
                UnitPrice = item[-3].strip()

                #Extract Line item - Total Price
                Price = item[-1].strip().replace(',','').replace('$','')
                
                DataFrame = DataFrame.append({'FileName':FileName,
                                'InvoiceNumber':InvoiceNumber,
                                'InvoiceDate':InvoiceDate,
                                'InvoiceTotal':InvoiceTotal,
                                'Quantity':Quantity,
                                'Desciption':Desciption,
                                'Price':Price},
                               ignore_index=True)
    else:
        print('[{}] Line item: Extraction Failed'.format(FileName))

    return (DataFrame)

def format2(data, FileName, DataFrame):
    #Extract Invoice Number
    start = data.find('Invoice no.') + len('Invoice no.') 
    end = data.find('Purchase Order')
    content = data[start:end]
    if len(content) > 0:
        content = content.split('\n')
        InvoiceNumber = content[0].strip()
    else:
        print('[{}] InvoiceNumber: Extraction Failed'.format(FileName))
        
    #Extract Invoice Date
    start = data.find('Invoice Date') + len('Invoice Date') 
    end = data.find('Terms')
    content = data[start:end]
    if len(content) > 0:
        content = content.split('\n')
        InvoiceDate = content[0].strip()
    else:
        print('[{}] InvoiceDate: Extraction Failed'.format(FileName))
        
    #Extract Invoice Total
    start = data.find('Invoice Amount') + len('Invoice Amount') 
    end = ''
    content = data[start:]
    if len(content) > 0:
        content = content.split('\n')
        InvoiceTotal = content[0].split(' ')[-1].replace(',','').replace('$','')
    else:
        print('[{}] InvoiceTotal: Extraction Failed'.format(FileName))
        
    #Extract Line Items
    start = data.find('Tax Amount') + len('Tax Amount') 
    end = data.find('Subtotal')
    content = data[start:end].replace('|','').replace(']','').replace('}','').replace('  ',' ')

    if len(content) > 0:
        content = content.split('\n')
        for item in content:
            if len(item) > 0:
                item = item.split(' ')

                #Extract Line item - Quantity
                Quantity = item[0].strip()

                #Extract Line item - Item No
                ItemNo = item[1].strip()

                #Extract Line item - Desciption
                Desciption = item[2:-3]
                Desciption = ' '.join(Desciption).strip()

                #Extract Line item - Unit Price
                UnitPrice = item[-3].strip()
                
                #Extract Line item - Total Price
                Price = item[-1].strip().replace(',','').replace('$','')
                
                DataFrame = DataFrame.append({'FileName':FileName,
                                'InvoiceNumber':InvoiceNumber,
                                'InvoiceDate':InvoiceDate,
                                'InvoiceTotal':InvoiceTotal,
                                'Quantity':Quantity,
                                'Desciption':Desciption,
                                'Price':Price},
                               ignore_index=True)
                
    else:
        print('[{}] Line item: Extraction Failed'.format(FileName))

    return (DataFrame)

In [3]:
#============================================
# Build transaction items with dataframe
#============================================

# create empty dataframe with fixed columns
list_column_names = ['FileName','InvoiceNumber','InvoiceDate','InvoiceTotal','Quantity','Desciption','Price'] 
df = pd.DataFrame(columns=list_column_names)

# directory to invoice files
FolderName = 'sample-invoices'
FolderPath = r"{}\{}".format(os.getcwd(),FolderName)

# iterate thru each invoice files
for FileName in os.listdir(FolderPath):
    if not FileName.endswith(".tiff"):
        continue
    path = os.path.join(FolderPath, FileName)
    if os.path.isdir(path):
        continue
        
    # convert image to text (formatted to remove empty line)
    data = pytesseract.image_to_string(path)
    data = ''.join([s for s in data.strip().splitlines(True) if s.strip()])
    
    # identity type of invoice format based using keywords, then extract required information from data accordingly
    if 'Ship to Invoice no.' in data:
        df = format1(data, FileName, df)
    else:
        df = format2(data, FileName, df)

df

,FileName,InvoiceNumber,InvoiceDate,InvoiceTotal,Quantity,Desciption,Price
0,Invoice1.tiff,10266,22 Aug 2018,1064.92,15,Sprocket - Large,577.50
1,Invoice1.tiff,10266,22 Aug 2018,1064.92,15,Sprocket - Med,284.25
2,Invoice1.tiff,10266,22 Aug 2018,1064.92,15,Sprocket - Small,133.50
3,Invoice10.tiff,10291,22 Aug 2018,3967.03,100,Tylenol Ex-Strong 100/box,1495.00
4,Invoice10.tiff,10291,22 Aug 2018,3967.03,150,NGK-ZFR5F-11 Spark Plug,412.50
5,Invoice10.tiff,10291,22 Aug 2018,3967.03,200,Radiator coolant,1000.00
6,Invoice10.tiff,10291,22 Aug 2018,3967.03,100,Snow White Paint,800.00
7,Invoice2.tiff,10267,22 Aug 2018,1198.40,25,Bracket - Widget Type A,181.25
8,Invoice2.tiff,10267,22 Aug 2018,1198.40,25,Bracket - Widget Type C,918.75
9,Invoice2.tiff,10267,22 Aug 2018,1198.40,25,Bracket - Widget Type B,20.00


In [7]:
#============================================
# Processing transactions item
#============================================

r.init(visual_automation = True)

# configurable variables - URL
url_InvoiceProcessing = 'https://developer.automationanywhere.com/challenges/automationanywherelabs-invoiceentry.html'

# configurable variables - CSS Selector/ XPaths in 'Invoice Processing' webpage
XPath_Button_AddItem = '//*[@id="myDIV"]/div/button[1]'

# configurable variables - HTML attributes in 'Invoice Processing' webpage
HTML_ID_invoiceNumber = 'invoiceNumber'
HTML_ID_invoiceDate = 'invoiceDate'
HTML_ID_invoiceTotal = 'invoiceTotal'

HTML_ID_quantity_StartWith = 'quantity_row_'
HTML_ID_desciption_StartWith = 'description_row_'
HTML_ID_price_StartWith = 'price_row_'

HTML_ID_fileupload = 'fileupload'
HTML_ID_TermsYes = 'agreeToTermsYes'
HTML_ID_submitbutton = 'submit_button'
HTML_ID_ResultTime = 'processing-time'
HTML_ID_ResultAccuracy = 'accuracy'

# configurable variables - filename of result
fileName = 'results_screenshot.png'

# access 'Invoice Processing' webpage
r.click('chrome.png')
r.wait(5)
r.url(url_InvoiceProcessing)

# initialise count for invoice file
invoice_count = 0
    
# Iterate thru each invoice file
for f in df['FileName'].unique():
    invoice_count = invoice_count + 1
    
    InvoiceNumber = df.loc[df['FileName'] == f]['InvoiceNumber'].values[0]
    InvoiceDate = df.loc[df['FileName'] == f]['InvoiceDate'].values[0]
    InvoiceTotal = df.loc[df['FileName'] == f]['InvoiceTotal'].values[0]
    r.type('//*[@id="{}"]'.format(HTML_ID_invoiceNumber),InvoiceNumber)
    r.type('//*[@id="{}"]'.format(HTML_ID_invoiceDate),InvoiceDate)
    r.type('//*[@id="{}"]'.format(HTML_ID_invoiceTotal),InvoiceTotal)
    
    # initialise count for line item
    item_count = 0
    
    # iterate thru each line items
    for row in df[(df['FileName']==f)].itertuples(index=False):
        item_count = item_count + 1
        
        # click on 'Add Item' button if more than one line items are detected.
        if item_count > 1:
            r.click(XPath_Button_AddItem)
        
        # type in Quantity, Description and Price per line item
        r.type('//*[@id="{}"]'.format(HTML_ID_quantity_StartWith+str(item_count)),row.Quantity)
        r.type('//*[@id="{}"]'.format(HTML_ID_desciption_StartWith+str(item_count)),row.Desciption)
        r.type('//*[@id="{}"]'.format(HTML_ID_price_StartWith+str(item_count)),row.Price)
    
    # click 'Choose File' button
    r.click(HTML_ID_fileupload)
    
    # copy and paste file path to invoice image to upload invoice details
    if invoice_count == 1:
        r.clipboard(os.path.join(FolderPath,f))
        r.keyboard('[ctrl]v'+'[enter]')
    else:
        r.keyboard(f.replace('.tiff','') + '[enter]')
    
    # click on 'Yes' radio button
    r.click(HTML_ID_TermsYes)
    
    # click on 'Submit' button
    r.click(HTML_ID_submitbutton)

# print results
Time = r.read('//*[@id="{}"]'.format(HTML_ID_ResultTime))
Accuracy = r.read('//*[@id="{}"]'.format(HTML_ID_ResultAccuracy))
if r.read('//*[@id="{}"]'.format(HTML_ID_ResultAccuracy)) != '100.00%':
    print ('Timing= {} / Accuracy= {}. Input error. Please try again'.format(Time, Accuracy))
else:
    print ('Timing= {} / Accuracy= {}'.format(Time, Accuracy))
    r.snap('page', fileName) #take screenshot of result
    
r.close()

Timing= 18.062 seconds / Accuracy= 100.00%


True